In [ ]:
var('t z v u')
var('r', domain='integer')

exponential_dict = {exp(2*k*r*log(2)) : 4^(k*r) for k in srange(1,5) + srange(-4,0)}

RBF = RealBallField(precision=500)
SCR = SR.subring(accepting_variables=(r,))
A.<n> = AsymptoticRing('QQ^n * n^QQ', SCR)
A_expansion.<Z> = AsymptoticRing('Z^QQ', SCR)

In [ ]:
catalan_asy_shifted = asymptotic_expansions.Binomial_kn_over_n(n, k=2, precision=5).subs({n: n-2}) / (n-1)
catalan_asy_shifted.show()

In [ ]:
z_loc = (1 - 1/Z)/4
assert 1 - 4*z_loc == Z^-1

In [ ]:
def T(z):
    return (1 - sqrt(1 - 4*z))/2

def S(z, t):
    return z + z*t/(1 - t - T(z)^2)

In [ ]:
S(z,z).taylor((z,0), 10)

In [ ]:
(z*T(z) + z).taylor((z, 0), 10)

## Section 3: Age of Catalan-Stanley Trees

In [ ]:
def F_main_factor(z):
    return T(z)^(2*r-1)/(1 + T(z)^(2*r-1))

def F_r_geq(z):
    return z * (1 + T(z)) * F_main_factor(z)

In [ ]:
F_main_factor(z_loc + O(Z^-3)).map_coefficients(lambda x: x.factor()
                                                .subs(exponential_dict).factor()).show()

In [ ]:
age_r_geq_expansion = F_r_geq(z_loc + O(Z^-3)).map_coefficients(lambda x: x.factor().subs(exponential_dict).factor())
age_r_geq_expansion.show()

In [ ]:
age_r_geq_asy = age_r_geq_expansion._singularity_analysis_(n, zeta=1/4)
age_r_geq_asy = age_r_geq_asy.map_coefficients(factor)
age_r_geq_asy.show()

In [ ]:
age_r_prob = (age_r_geq_asy - age_r_geq_asy.map_coefficients(lambda x: x.subs({r: r+1}))) / catalan_asy_shifted
age_r_prob = age_r_prob.map_coefficients(lambda x: x.subs(exponential_dict).factor())
age_r_prob.show()

In [ ]:
age_r_expectation = age_r_geq_asy / catalan_asy_shifted
age_r_expectation = age_r_expectation.map_coefficients(lambda x: x.subs(exponential_dict).factor())
age_r_expectation.show()

In [ ]:
c0 = RBF(sum(age_r_expectation.monomial_coefficient(A(1)), r, 1, 100))
c0

In [ ]:
c1 = RBF(sum(age_r_expectation.monomial_coefficient(n^-1), r, 1, 100))
c1

In [ ]:
c2 = RBF(sum((2*r-1)*age_r_expectation.monomial_coefficient(A(1)), r, 1, 100)) - c0^2
c2

In [ ]:
c3 = RBF(sum((2*r-1)*age_r_expectation.monomial_coefficient(n^-1), r, 1, 100)) - 2*c0*c1
c3

In [ ]:
F_r_geq(u/(1+u)^2).factor().subs({
    sqrt((u^2 - 2*u  +1)/(u+1)^2): (1-u)/(1+u),
    1/sqrt((u^2 - 2*u  +1)/(u+1)^2): (1+u)/(1-u),
    ((u^2 - 2*u  +1)/(u+1)^2)^(3/2): (1-u)^3/(1+u)^3,
    ((u^2 - 2*u  +1)/(u+1)^2)^(5/2): (1-u)^5/(1+u)^5
}).factor().show()

## Section 4: Analysis of Younger Generations

In [ ]:
def G_r(z,v):
    B = (1 - z*(1-T(z)^(2*r))/(1 - T(z)^2))
    return z*v/(B - z*v*T(z)^(2*r)/(1 - T(z*v)^2))

In [ ]:
size_expectation_expansion = fast_callable(G_r(z,v).diff(v,1)(z=z, v=1), vars=(z,r))(z_loc + O(Z^-5), r)
size_expectation_expansion = size_expectation_expansion.map_coefficients(lambda x: x.factor().subs(exponential_dict).factor())
size_expectation_expansion.map_coefficients(factor).show()

In [ ]:
size_expectation_asy = size_expectation_expansion._singularity_analysis_(n, zeta=1/4) / catalan_asy_shifted
size_expectation_asy = size_expectation_asy.map_coefficients(factor)
size_expectation_asy.show()

In [ ]:
size_fmoment_2_expansion = fast_callable(G_r(z,v).diff(v,2)(z=z, v=1), vars=(z,r))(z_loc + O(Z^-5), r)
size_fmoment_2_expansion = size_fmoment_2_expansion.map_coefficients(lambda x: x.factor().subs(exponential_dict).factor())
size_fmoment_2_expansion.show()

In [ ]:
size_fmoment_2_asy = size_fmoment_2_expansion._singularity_analysis_(n, zeta=1/4) / catalan_asy_shifted
size_fmoment_2_asy = size_fmoment_2_asy.map_coefficients(factor)
size_fmoment_2_asy.show()

In [ ]:
size_variance_asy = size_fmoment_2_asy + size_expectation_asy - size_expectation_asy^2
size_variance_asy = size_variance_asy.map_coefficients(factor)
size_variance_asy.show()

In [ ]:
G_r(z,v).diff(v,1)(z=u/(1+u)^2,v=1).factor().subs({
    sqrt((u^2 - 2*u  +1)/(u+1)^2): (1-u)/(1+u),
    1/sqrt((u^2 - 2*u  +1)/(u+1)^2): (1+u)/(1-u),
    ((u^2 - 2*u  +1)/(u+1)^2)^(3/2): (1-u)^3/(1+u)^3,
    ((u^2 - 2*u  +1)/(u+1)^2)^(5/2): (1-u)^5/(1+u)^5
}).factor().show()